In [10]:
import pandas as pd
import numpy as np
import gc
from datetime import datetime
CHUNK_SIZE = 10000

start = datetime.now()

reader_date = pd.read_csv("train_date.csv",chunksize=CHUNK_SIZE, dtype=np.float32)
reader_num = pd.read_csv("train_numeric.csv",chunksize=CHUNK_SIZE, dtype=np.float32)

#分割して読み込み
loop = 0
cnt_0 = 0
cnt_1 = 0
for df_chunk_date in reader_date:
    df_chunk_num = reader_num.get_chunk(CHUNK_SIZE)
    
    df_chunk = pd.merge(on="Id", left=df_chunk_date, right=df_chunk_num)
    cnt_0 = cnt_0 + len(df_chunk[df_chunk['Response'] == 0].index)
    cnt_1 = cnt_1 + len(df_chunk[df_chunk['Response'] == 1].index)

    if loop == 0:
        df = df_chunk
    else:
        df = pd.concat([df, df_chunk])
    
    print("loop : {} / Rows = {}".format(loop, len(df.index)))

    loop = loop + 1

print("0:{}, 1:{}".format(cnt_0, cnt_1))
print("{} -> {}".format(start, datetime.now()))
print("Memory:{}".format(df.memory_usage(index=True).sum()))

loop : 0 / Rows = 1113
loop : 1 / Rows = 2352
loop : 2 / Rows = 3549
loop : 3 / Rows = 4599
loop : 4 / Rows = 5691
loop : 5 / Rows = 6783
loop : 6 / Rows = 8064
loop : 7 / Rows = 9303
loop : 8 / Rows = 10794
loop : 9 / Rows = 11928
loop : 10 / Rows = 13167
loop : 11 / Rows = 14280
loop : 12 / Rows = 15078
loop : 13 / Rows = 16380
loop : 14 / Rows = 17535
loop : 15 / Rows = 18543
loop : 16 / Rows = 20118
loop : 17 / Rows = 21546
loop : 18 / Rows = 22617
loop : 19 / Rows = 23709
loop : 20 / Rows = 24864
loop : 21 / Rows = 25809
loop : 22 / Rows = 26670
loop : 23 / Rows = 27825
loop : 24 / Rows = 29610
loop : 25 / Rows = 30744
loop : 26 / Rows = 31899
loop : 27 / Rows = 33348
loop : 28 / Rows = 34566
loop : 29 / Rows = 35595
loop : 30 / Rows = 36876
loop : 31 / Rows = 38262
loop : 32 / Rows = 39501
loop : 33 / Rows = 40803
loop : 34 / Rows = 42021
loop : 35 / Rows = 43596
loop : 36 / Rows = 44625
loop : 37 / Rows = 45822
loop : 38 / Rows = 47334
loop : 39 / Rows = 48636
loop : 40 / Rows =

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split

start = datetime.now()

## 教師データと訓練データに分離
y = df['Response']
X = df.drop(['Id', 'Response'],axis=1)
X = X.fillna(99999)

## テストセットと訓練セットに分離
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.head())
print(y_train.head())

print(X_test.head())
print(y_test.head())

print("{} -> {}".format(start, datetime.now()))

In [4]:
## 学習
import lightgbm as lgb

start = datetime.now()

clf = lgb.LGBMClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
print(X_train.shape)
print("{} -> {}".format(start, datetime.now()))

(793110, 2124)
2018-07-12 22:37:55.518885 -> 2018-07-12 22:40:41.746190


In [5]:
from sklearn import metrics

start = datetime.now()


y_pred = clf.predict(X_test.values)
print(y_test.values)
print(y_pred)
print("matthews_corrcoef = {}".format(metrics.matthews_corrcoef(y_pred, y_test.values)))
df_pred = pd.DataFrame({'Response':y_pred})
num_0 = len(df_pred[df_pred['Response'] == 0].index)
num_1 = len(df_pred[df_pred['Response'] == 1].index)
print("pred : 1:{} , 0:{} = {}".format(num_1, num_0, num_1 / num_0))
num_0 = len(y_test[y_test == 0].index)
num_1 = len(y_test[y_test == 1].index)
print("test : 1:{} / 0:{} = {}".format(num_1, num_0, num_1 / num_0))
print("{} -> {}".format(start, datetime.now()))

C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
matthews_corrcoef = 0.1612155449364572
pred : 1:140 , 0:390497 = 0.0003585174790075212
test : 1:2299 / 0:388338 = 0.005920100530980744
2018-07-12 22:40:58.362963 -> 2018-07-12 22:41:10.863386


In [6]:
start = datetime.now()

#特徴量重要度を表示
for i in range(0, X_train.shape[1]):
    if 0 < clf.feature_importances_[i]:
        print("{}:{}".format(X_train.columns[i], clf.feature_importances_[i]))
print("{} -> {}".format(start, datetime.now()))

L1_S24_D999:1
L1_S24_D1522:1
L1_S24_D1558:4
L1_S24_D1576:1
L1_S24_D1674:1
L1_S25_D2406:1
L1_S25_D2445:1
L1_S25_D2596:1
L3_S29_D3316:3
L3_S29_D3474:11
L3_S30_D3496:1
L3_S30_D3566:2
L3_S32_D3852:5
L3_S33_D3856:11
L3_S34_D3875:14
L3_S36_D3919:1
L3_S37_D3942:10
L3_S38_D3953:18
L3_S49_D4208:1
L0_S0_F2:2
L0_S0_F6:3
L0_S0_F8:1
L0_S0_F18:1
L0_S0_F20:1
L0_S2_F36:1
L0_S2_F48:1
L0_S3_F100:1
L0_S4_F104:3
L0_S4_F109:3
L0_S10_F274:1
L0_S13_F354:1
L0_S14_F390:1
L0_S16_F426:2
L0_S21_F472:1
L0_S21_F497:1
L0_S22_F546:1
L0_S22_F566:1
L0_S22_F591:1
L0_S23_F619:1
L0_S23_F623:1
L0_S23_F647:1
L0_S23_F671:2
L1_S24_F829:1
L1_S24_F839:3
L1_S24_F925:1
L1_S24_F1102:1
L1_S24_F1126:1
L1_S24_F1172:1
L1_S24_F1406:3
L1_S24_F1482:1
L1_S24_F1494:3
L1_S24_F1498:7
L1_S24_F1567:1
L1_S24_F1604:6
L1_S24_F1609:4
L1_S24_F1632:1
L1_S24_F1695:11
L1_S24_F1700:2
L1_S24_F1713:3
L1_S24_F1753:1
L1_S24_F1768:1
L1_S24_F1829:1
L1_S24_F1844:2
L1_S24_F1846:11
L1_S25_F2046:1
L1_S25_F2390:1
L1_S25_F2672:1
L1_S25_F2773:1
L1_S25_F3030:1
L2_S2

In [7]:
start = datetime.now()

#全訓練データを用いて再学習
clf.fit(X, y)
print("{} -> {}".format(start, datetime.now()))

MemoryError: 

In [8]:
start = datetime.now()
reader_test_date = pd.read_csv("test_date.csv",chunksize=CHUNK_SIZE)
reader_test_num = pd.read_csv("test_numeric.csv",chunksize=CHUNK_SIZE)

#分割して読み込み
loop = 0
for df_chunk_test_date in reader_test_date:
    df_chunk_test_num = reader_test_num.get_chunk(CHUNK_SIZE)
    
    df_test_chunk = pd.merge(on="Id", left=df_chunk_test_date, right=df_chunk_test_num)
    df_test_id_chunk = df_test_chunk['Id']
    df_test_chunk = df_test_chunk.drop('Id',axis=1)
    
    ##予測
    ary_test_chunk_pred = clf.predict(df_test_chunk.values)
    df_test_chunk_pred = pd.DataFrame({'Id':df_test_id_chunk.values, 'Response':ary_test_chunk_pred})
    if loop == 0:
        df_test_pred = df_test_chunk_pred
    else:
        df_test_pred = pd.concat([df_test_pred, df_test_chunk_pred])
    
    print("loop : {} / Rows = {}".format(loop, len(df_test_pred.index)))
    
    loop = loop + 1

##CSV出力
y_submit = df_test_pred
print(y_submit)
print("y_submit : {}".format(y_submit.shape))
y_submit.to_csv("submit_chunk.csv", columns=['Id', 'Response'], index=False)
print("'submit_chunk.csv' is saved.")
print("{} : {}".format(len(y_submit[y_submit['Response'] == 0].index), len(y_submit[y_submit['Response'] == 1].index)))

print("{} -> {}".format(start, datetime.now()))

C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


loop : 0 / Rows = 10000


C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


loop : 1 / Rows = 20000


C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


...

loop : 116 / Rows = 1170000


C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


loop : 117 / Rows = 1180000


C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


loop : 118 / Rows = 1183748
           Id  Response
0           1       0.0
1           2       1.0
2           3       1.0
3           5       1.0
4           8       1.0
5          10       1.0
6          12       1.0
7          15       1.0
8          17       1.0
9          19       1.0
10         20       1.0
11         21       1.0
12         22       1.0
13         24       1.0
14         25       1.0
15         29       1.0
16         30       1.0
17         32       1.0
18         33       1.0
19         35       1.0
20         36       1.0
21         37       1.0
22         39       0.0
23         40       1.0
24         42       1.0
25         43       0.0
26         45       1.0
27         46       1.0
28         48       1.0
29         50       1.0
...       ...       ...
3718  2367436       1.0
3719  2367438       1.0
3720  2367441       1.0
3721  2367445       1.0
3722  2367446       1.0
3723  2367447       0.0
3724  2367448       1.0
3725  2367449       1.0
3726  236745